In [ ]:
!pip install scipy scikit-image torch tqdm transformers mtcnn mediapipe opencv-python torchvision numpy pandas timm evaluate gluoncv mxnet 

In [2]:
from google.colab import drive
from pathlib import Path
drive.mount("/content/drive")
project_path = Path("/content/drive/MyDrive/NLP/MultiModalEmotionRecognition/")

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/NLP/MultiModalEmotionRecognition/data

/content/drive/MyDrive/NLP/MultiModalEmotionRecognition/data


In [4]:
!ls

correct_indexes   error_indexes		saved_features	    train_ende.zip
dev.zip		  image_index_test.txt	sentiment_test.txt
english_test.txt  image_index_val.txt	sentiment_val.txt
english_val.txt   images		test.zip


In [ ]:
!unzip dev.zip
!unzip test.zip
!unzip train_ende.zip

In [ ]:
%mv dev/ images/val
%mv test/ images/test
%mv train_ende/ images/train

In [ ]:
%ls images/test -1 | wc -l

5067


In [6]:
%cd ..

/content/drive/MyDrive/NLP/MultiModalEmotionRecognition


In [8]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [9]:
import torch
import numpy as np
from gluoncv import model_zoo, data
from gluoncv.data.transforms.pose import detector_to_simple_pose, heatmap_to_coord

class PoseEmbeddingExtractor:
    def __init__(
        self,
    ):
        self.detector = model_zoo.get_model('yolo3_mobilenet1.0_coco', pretrained=True)
        self.pose_net = model_zoo.get_model('simple_pose_resnet18_v1b', pretrained=True)
        self.detector.reset_class(["person"], reuse_weights=['person'])

    def detect_person(self, x, image):
        class_IDs, scores, bounding_boxs = self.detector(x)
        pose_input, upscale_bbox = detector_to_simple_pose(image, class_IDs, scores, bounding_boxs)
        return pose_input, upscale_bbox

    def get_most_confident_coords(self, predicted_heatmap, upscale_bbox):
        pred_coords, confidence = heatmap_to_coord(predicted_heatmap, upscale_bbox)
        mean_confidence = np.mean(confidence[:,:,0].asnumpy(), axis=1)
        best_confidence_arg = mean_confidence.argmax()
        best_coords = pred_coords[best_confidence_arg].asnumpy().ravel()
        return best_coords

    def extract_embedding(self, image_path):
        x, image = data.transforms.presets.ssd.load_test(image_path, short=512)
        pose_input, upscale_bbox = self.detect_person(x, image)
        predicted_heatmap = self.pose_net(pose_input)
        best_coords = self.get_most_confident_coords(predicted_heatmap, upscale_bbox)
        return torch.tensor(best_coords)


/usr/local/lib/python3.7/dist-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.9.1` and `torch==1.12.1+cu113` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


#Face Embedding

In [28]:
from scipy.spatial.distance import euclidean
import math
from skimage.transform import rotate
from mtcnn import MTCNN
import mediapipe
import numpy as np
import pandas as pd
import cv2
import os
from PIL import Image
import torch
from torchvision import transforms
import urllib


def get_model_path(model_name):
    model_file = model_name + ".pt"
    cache_dir = os.path.join(os.path.expanduser("~"), ".hsemotions")
    # cache_dir = "emotion_models"
    os.makedirs(cache_dir, exist_ok=True)
    fpath = os.path.join(cache_dir, model_file)
    if not os.path.isfile(fpath):
        print(f"{model_file} not exists")
        url = (
            "https://github.com/HSE-asavchenko/face-emotion-recognition/blob/main/models/affectnet_emotions/"
            + model_file
            + "?raw=true"
        )
        print("Downloading", model_name, "from", url)
        urllib.request.urlretrieve(url, fpath)

    return fpath


class FaceAlignment:
    def __init__(
        self,
    ):
        pass

    @staticmethod
    def apply_rotation_on_images(input_images, angles):
        rotated_images = [
            rotate(image, angle) for image, angle in zip(input_images, angles)
        ]
        return rotated_images

    @staticmethod
    def compute_alignment_rotation_(eyes_coordinates):
        angles = []
        directions = []
        for left_eye_coordinate, right_eye_coordinate in eyes_coordinates:

            left_eye_x, left_eye_y = left_eye_coordinate
            right_eye_x, right_eye_y = right_eye_coordinate

            triangle_vertex = (
                (right_eye_x, left_eye_y)
                if left_eye_y > right_eye_y
                else (left_eye_x, right_eye_y)
            )
            direction = (
                -1 if left_eye_y > right_eye_y else 1
            )  # rotate clockwise else counter-clockwise

            # compute length of triangle edges
            a = euclidean(left_eye_coordinate, triangle_vertex)
            b = euclidean(right_eye_coordinate, triangle_vertex)
            c = euclidean(right_eye_coordinate, left_eye_coordinate)

            # cosine rule
            if (
                b != 0 and c != 0
            ):  # this multiplication causes division by zero in cos_a calculation
                cos_a = (b**2 + c**2 - a**2) / (2 * b * c)
                angle = np.arccos(cos_a)  # angle in radian
                angle = (angle * 180) / math.pi  # radian to degree
            else:
                angle = 0

            angle = angle - 90 if direction == -1 else angle

            angles.append(angle)
            directions.append(direction)

        return angles, directions


class FaceDetection:

    # first call extract_face
    def __init__(self, model_name, minimum_confidence):

        self.detected_faces_information = None
        self.model_name = model_name
        self.minimum_confidence = minimum_confidence

        if model_name == "MTCNN":
            detector_model = MTCNN()
            self.detect_faces_function = (
                lambda input_image: detector_model.detect_faces(input_image)
            )

    def extract_faces(self, input_image, return_detections_information=True):
        self.detect_faces__(input_image)
        faces = self.get_faces__(
            input_image,
        )
        if return_detections_information:
            return faces, self.detected_faces_information

        else:
            return faces

    def detect_faces__(self, input_image):
        detections = self.detect_faces_function(input_image)
        self.detected_faces_information = list(
            filter(
                lambda element: element["confidence"] > self.minimum_confidence,
                detections,
            )
        )

    def get_detected_faces_information(self):
        return self.detected_faces_information

    def get_keypoints(
        self,
    ):
        return list(
            map(lambda element: element["keypoints"], self.detected_faces_information)
        )

    def get_faces__(
        self,
        input_image,
    ):
        boxes = [
            detection_information["box"]
            for detection_information in self.detected_faces_information
        ]
        y1y2x1x2 = [(int(y), int(y + h), int(x), int(x + w)) for x, y, w, h in boxes]
        faces = [input_image[y1:y2, x1:x2] for y1, y2, x1, x2 in y1y2x1x2]
        return faces

    def get_eyes_coordinates(
        self,
    ):
        eyes_coordinates = [
            (info["keypoints"]["left_eye"], info["keypoints"]["right_eye"])
            for info in self.detected_faces_information
        ]
        return eyes_coordinates


class FaceEmotionRecognizer:
    # supported values of model_name: enet_b0_8_best_vgaf, enet_b0_8_best_afew, enet_b2_8, enet_b0_8_va_mtl, enet_b2_7
    def __init__(self, device, model_name="enet_b0_8_best_vgaf"):
        self.device = device
        self.is_mtl = "_mtl" in model_name
        if "_7" in model_name:
            self.idx_to_class = {
                0: "Anger",
                1: "Disgust",
                2: "Fear",
                3: "Happiness",
                4: "Neutral",
                5: "Sadness",
                6: "Surprise",
            }
        else:
            self.idx_to_class = {
                0: "Anger",
                1: "Contempt",
                2: "Disgust",
                3: "Fear",
                4: "Happiness",
                5: "Neutral",
                6: "Sadness",
                7: "Surprise",
            }

        self.img_size = 224 if "_b0_" in model_name else 260
        self.test_transforms = transforms.Compose(
            [
                transforms.Resize((self.img_size, self.img_size)),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )

        path = get_model_path(model_name)

        model = torch.load(path)
        model = model.to(device)

        if isinstance(model.classifier, torch.nn.Sequential):
            self.classifier_weights = model.classifier[0].weight.data
            self.classifier_bias = model.classifier[0].bias.data
        else:
            self.classifier_weights = model.classifier.weight.data
            self.classifier_bias = model.classifier.bias.data

        model.classifier = torch.nn.Identity()
        self.model = model.eval()
        # print(path, self.test_transforms)

    def compute_probability(self, features):
        return torch.matmul(features, self.classifier_weights.T) + self.classifier_bias

    def extract_representations_from_faces(self, input_faces):
        faces = [self.test_transforms(Image.fromarray(face)) for face in input_faces]
        features = self.model(torch.stack(faces, dim=0).to(self.device))
        return features

    def predict_emotions_from_representations(
        self, representations, logits=True, return_features=True
    ):
        scores = self.compute_probability(representations)
        if self.is_mtl:
            predictions_indices = torch.argmax(scores[:, :-2], dim=1)

        else:
            predictions_indices = torch.argmax(scores, dim=1)

        if self.is_mtl:
            x = scores[:, :-2]

        else:
            x = scores
        pred = torch.argmax(x[0])

        if not logits:
            e_x = torch.exp(x - torch.max(x, dim=1)[:, None])
            e_x = e_x / e_x.sum(dim=1)[:, None]
            if self.is_mtl:
                scores[:, :-2] = e_x
            else:
                scores = e_x

        return [
            self.idx_to_class[pred.item()] for pred in (predictions_indices)
        ], scores


class FaceNormalizer:
    def __init__(self):
        self.mp_face_mesh = mediapipe.solutions.face_mesh
        face_mesh = self.mp_face_mesh.FaceMesh(static_image_mode=True)

        mp_face_mesh = mediapipe.solutions.face_mesh
        self.face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)
        self.routes_idx = self.initialize__()

    def initialize__(self):
        df = pd.DataFrame(
            list(self.mp_face_mesh.FACEMESH_FACE_OVAL), columns=["p1", "p2"]
        )
        routes_idx = []

        p1 = df.iloc[0]["p1"]
        p2 = df.iloc[0]["p2"]

        for i in range(0, df.shape[0]):
            obj = df[df["p1"] == p2]
            p1 = obj["p1"].values[0]
            p2 = obj["p2"].values[0]

            route_idx = []
            route_idx.append(p1)
            route_idx.append(p2)
            routes_idx.append(route_idx)

        return routes_idx

    def get_landmarks__(self, input_image: np.ndarray):
        if input_image.dtype == np.float:
            input_image = (input_image * 255).astype(np.uint8)

        results = self.face_mesh.process(input_image)
        landmarks = results.multi_face_landmarks[0]

        routes = []
        # for source_idx, target_idx in mp_face_mesh.FACEMESH_FACE_OVAL:
        for source_idx, target_idx in self.routes_idx:
            source = landmarks.landmark[source_idx]
            target = landmarks.landmark[target_idx]

            relative_source = (
                int(input_image.shape[1] * source.x),
                int(input_image.shape[0] * source.y),
            )
            relative_target = (
                int(input_image.shape[1] * target.x),
                int(input_image.shape[0] * target.y),
            )

            # cv2.line(img, relative_source, relative_target, (255, 255, 255), thickness = 2)

            routes.append(relative_source)
            routes.append(relative_target)

        return routes

    @staticmethod
    def normalize_with_landmark_points__(input_image, landmarks):
        mask = np.zeros((input_image.shape[0], input_image.shape[1]))
        mask = cv2.fillConvexPoly(mask, np.array(landmarks), 1)
        mask = mask.astype(bool)

        out = np.zeros_like(input_image)
        out[mask] = input_image[mask]
        return out

    def normalize_faces_image(self, input_images):
        normalized_faces_images = [
            self.normalize_with_landmark_points__(
                input_image, self.get_landmarks__(input_image)
            )
            for input_image in input_images
        ]
        return normalized_faces_images


class FaceEmbeddingExtractor:
    def __init__(
        self,
    ):
        self.face_detection_model: FaceDetection = None
        self.face_alignment_model: FaceAlignment = None
        self.face_normalizer_model: FaceNormalizer = None
        self.face_emotion_recognition_model: FaceEmotionRecognizer = None

        self.faces = None
        self.normalized_rotated_faces = None
        self.rotated_faces = None
        self.rotation_angles = None
        self.rotation_directions = None

    def set_face_detection_model(self, face_detection_model):
        self.face_detection_model = face_detection_model
        return self

    def set_face_alignment_model(self, face_alignment_model):
        self.face_alignment_model = face_alignment_model
        return self

    def set_face_normalizer_model(self, face_normalizer_model):
        self.face_normalizer_model = face_normalizer_model
        return self

    def set_face_emotion_recognition_model(self, face_emotion_recognition_model):
        self.face_emotion_recognition_model = face_emotion_recognition_model
        return self

    def extract_embedding(self, input_image):
        faces, detected_faces_information = self.face_detection_model.extract_faces(
            input_image, return_detections_information=True
        )

        (
            rotation_angles,
            rotation_directions,
        ) = self.face_alignment_model.compute_alignment_rotation_(
            self.face_detection_model.get_eyes_coordinates()
        )
        rotated_faces = self.face_alignment_model.apply_rotation_on_images(
            faces, rotation_angles
        )
        normalized_rotated_faces = self.face_normalizer_model.normalize_faces_image(
            rotated_faces
        )

        normalized_rotated_faces_255 = [
            (image * 255).astype(np.uint8) for image in normalized_rotated_faces
        ]

        representations = (
            self.face_emotion_recognition_model.extract_representations_from_faces(
                normalized_rotated_faces_255
            )
        )[0] #WARNING: 0 was not here
        del normalized_rotated_faces_255
        del normalized_rotated_faces
        del rotated_faces
        del rotation_angles
        del rotation_directions
        del faces
        del detected_faces_information
        # (
        #     predictions,
        #     scores,
        # ) = self.face_emotion_recognition_model.predict_emotions_from_representations(
        #     representations
        # )

        # self.faces = faces
        # self.rotation_angles, self.rotation_directions = (
        #     rotation_angles,
        #     rotation_directions,
        # )
        # self.rotated_faces = rotated_faces
        # self.normalized_rotated_faces = normalized_rotated_faces_255

        return None, None, representations
        # return preictions, scores, representations

    def get_rotations_information(self):
        return self.rotation_angles, self.rotation_directions

    def get_faces(self):
        return self.faces

    def get_rotated_faces(self):
        return self.rotated_faces

    def get_normalized_rotated_faces(self):
        return self.normalized_rotated_faces

    def clear(self):
        self.faces = None
        self.normalized_rotated_faces = None
        self.rotated_faces = None
        self.rotation_angles = None
        self.rotation_directions = None

    def store_embeddings(self, file, embeddings):
        with open(file, "wb") as file_out:
            pickle.dump(
                {"embeddings": embeddings}, file_out, protocol=pickle.HIGHEST_PROTOCOL
            )

    def load_embeddings(self, file):
        with open(file, "rb") as file_in:
            stored_data = pickle.load(file_in)
            stored_embeddings = stored_data["embeddings"]

        return stored_embeddings

#Text Embedding


In [29]:
from transformers import AutoTokenizer, AutoModel, pipeline
from transformers import RobertaForSequenceClassification
import torch
import pickle


class TextEmbeddingExtractor:
    def __init__(
        self,
        model_name="pysentimiento/robertuito-sentiment-analysis",
        batch_size=250,
        show_progress_bar=True,
        to_tensor=True,
        max_length=128,
    ):
        self.model_name = model_name

        self.device = device

        self.batch_size = batch_size
        self.show_progress_bar = show_progress_bar
        self.to_tensor = to_tensor

        self.max_length = max_length

        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        # self.model = AutoModel.from_pretrained(self.model_name).to(self.device)

        self.model = RobertaForSequenceClassification.from_pretrained(
            self.model_name, num_labels=3, output_hidden_states=True
        ).to(self.device)

        # C1
        self.generator = pipeline(
            task="sentiment-analysis",
            model=self.model,
            tokenizer=self.tokenizer,
        )

    def extract_embedding(
        self,
        input_batch_sentences,
    ):
        encoded_input = self.tokenizer(
            input_batch_sentences,
            padding=True,
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        ).to(self.device)

        with torch.no_grad():
            model_output = self.model(**encoded_input)
            hidden_states = model_output["hidden_states"]
            last_layer_hidden_states = hidden_states[
                12
            ]  # 12 = len(hidden_states) , dim = (batch_size, seq_len, 768)
            cls_hidden_state = last_layer_hidden_states[:, 0, :]

        return cls_hidden_state

    def get_labels(self, input_batch_sentences):
        return self.generator(input_batch_sentences)

    @staticmethod
    def store_embeddings(file, embeddings):
        with open(file, "wb") as file_out:
            pickle.dump(
                {"embeddings": embeddings}, file_out, protocol=pickle.HIGHEST_PROTOCOL
            )

    @staticmethod
    def load_embeddings(file):
        with open(file, "rb") as file_in:
            stored_data = pickle.load(file_in)
            stored_embeddings = stored_data["embeddings"]

        return stored_embeddings

#Dataset

In [30]:
!free

              total        used        free      shared  buff/cache   available
Mem:       13298580     7093508      164564       21704     6040508     9269192
Swap:             0           0           0


In [35]:
import os, cv2, torch, ast
import pandas as pd
import numpy as np
from pathlib import Path
from torch.utils.data import Dataset
from tqdm import trange
from tqdm import tqdm


class MSCTDDataSet(Dataset):
    """MSCTD dataset."""

    def __init__(self, base_path="data/", dataset_type="train", data_size=None, load=False, raw=False):
        """
        Args:
            base_path (str or path): path to data folder
            dataset_type (str): dev, train, test
        """
        if isinstance(base_path, str):
            base_path = Path(base_path)
        self.base_path = base_path
        self.load_path = base_path / 'saved_features'
        self.dataset_type = dataset_type
        self.text_file_path = base_path / f"english_{dataset_type}.txt"
        self.seq_file_path = base_path / f"image_index_{dataset_type}.txt"
        self.sentiment_file_path = base_path / f"sentiment_{dataset_type}.txt"
        self.image_dir = base_path / "images" / dataset_type
        self.correct_indexes_file_path = base_path / "correct_indexes" / f"correct_indexes_{dataset_type}.txt"

        self.data_size = data_size
        self.load = load
        self.raw = raw

        self.texts = None
        self.sentiments = None
        self.indexes = None
        self.face_embeddings = None
        self.pose_embeddings = None
        self.text_embeddings = None
        self.load_data()
        self.face_embedding_extractor = self.get_face_embedding_extractor()
        self.text_embedding_extractor = TextEmbeddingExtractor()
        self.pose_embedding_extractor = PoseEmbeddingExtractor()


    def get_face_embedding_extractor(self):
        fd = FaceDetection("MTCNN", minimum_confidence=0.95)
        fa = FaceAlignment()
        fn = FaceNormalizer()
        model_name = "enet_b0_8_best_afew"
        fer = FaceEmotionRecognizer(device, model_name)
        fre = (
            FaceEmbeddingExtractor()
            .set_face_detection_model(fd)
            .set_face_alignment_model(fa)
            .set_face_normalizer_model(fn)
            .set_face_emotion_recognition_model(fer)
        )
        return fre

    def load_data(self):
        with open(self.text_file_path) as text_file, open(self.sentiment_file_path) as sentiment_file, open(self.correct_indexes_file_path) as correct_file:
            texts = [t.strip() for t in text_file.readlines()]
            sentiments = [int(t.strip()) for t in sentiment_file.readlines()]
            face_embeddings = None
            pose_embeddings = None
            text_embeddings = None
            corrects = [int(c.strip()) for c in correct_file.readlines()]
            if self.load:
                try:
                    face_embeddings = torch.load(self.save_path / f'face_embeddings_{self.dataset_type}.pt')
                    pose_embeddings = torch.load(self.save_path / f'pose_embeddings_{self.dataset_type}.pt')
                    text_embeddings = torch.load(self.save_path / f'text_embeddings_{self.dataset_type}.pt')
                    corrects = torch.load(self.save_path / f'real_indexes_{self.dataset_type}.pt')
                except Exception as e:
                    print(e)
                    print('Warning: passed load=True but not embedding file was located. Not loading')

            correct_texts = [texts[i] for i in corrects]
            correct_sentiments = [sentiments[i] for i in corrects]
        # with open(self.image_index_path) as f:
        #     images = [ast.literal_eval(t.strip()) for t in f.readlines()]

        if self.data_size:
            correct_texts = correct_texts[: self.data_size]
            correct_sentiments =correct_sentiments[: self.data_size]
            if face_embeddings:
                face_embeddings = face_embeddings[:self.data_size,:]
            if pose_embeddings:
                pose_embeddings = pose_embeddings[:self.data_size,:]
            if text_embeddings:
                face_embeddings = text_embeddings[:self.data_size,:]
            # images = images[: self.data_size]


        self.texts = correct_texts
        self.text_embeddings = text_embeddings
        self.sentiments = correct_sentiments
        self.indexes = corrects
        self.face_embeddings = face_embeddings
        self.pose_embeddings = pose_embeddings



    def __len__(self):
        return len(self.texts)

    def get_face_embedding(self, index, image):
        if self.load:
            print('loading img')
            return self.face_embeddings[index]
        (
            predictions,
            scores,
            representations,
        ) = self.face_embedding_extractor.extract_embedding(image)
        return representations

    def get_pose_embedding(self, index, image):
        if self.load:
            print('loading img')
            return self.pose_embeddings[index]
        return self.pose_embedding_extractor.extract_embedding(image)

    def get_image_embeddings(self, index):
        image = None
        real_index = self.indexes[index]
        image_name = self.image_dir / f"{real_index}.jpg"
        if not self.load:
            image = cv2.imread(str(image_name))[:, :, ::-1]

        face_embedding = self.get_face_embedding(index, image)
        pose_embedding = self.get_pose_embedding(index, str(image_name))
        return face_embedding, pose_embedding

    def get_sentiment(self, index):
        return self.sentiments[index]



    def get_text(self, index):
        if self.load and not self.text_embeddings is None:
            print('loading txt')
            return self.text_embeddings[index]
        text = self.texts[index]
        text = self.text_embedding_extractor.extract_embedding([text])[0]
        return text

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        
        face_embedding, pose_embedding = self.get_image_embeddings(index)
        sentiment = self.get_sentiment(index)
        text = self.get_text(index)
        sample = {"real_index": self.indexes[index], "pose_embedding": pose_embedding, "face_embedding": face_embedding, "text_embedding": text, "sentiment": sentiment}
        if self.raw:
            sample["text"] = self.texts[index]
            # sample["image"] = pass 

        return sample


In [36]:
# %mkdir data/saved_features/

In [37]:
from torch.utils.data import DataLoader

ds = MSCTDDataSet(base_path=project_path / "data/", dataset_type = "val", load=False)
print(len(ds))
dataloader = DataLoader(ds, batch_size=8)

3461


In [ ]:
def save_features(dataloader, dataset_type):
    save_path = project_path / 'data' / 'saved_features'
    # stop_batch = 6

    for batch_index, batch in enumerate(tqdm(dataloader)):
        # print(batch["face_embedding"].shape)
        # print(batch["text_embedding"].shape)
        # print(batch["real_index"].shape)
        torch.save(batch["face_embedding"], save_path / f'face_embeddings_{dataset_type}_{batch_index}.pt')
        torch.save(batch["pose_embedding"], save_path / f'pose_embeddings_{dataset_type}_{batch_index}.pt')
        torch.save(batch["text_embedding"], save_path / f'text_embeddings_{dataset_type}_{batch_index}.pt')
        torch.save(batch["real_index"], save_path / f'real_indexes_{dataset_type}_{batch_index}.pt')
        # if batch_index==stop_batch:
        #   break

    print('----------------------')
    print(len(dataloader))
    len_batch = len(dataloader)
    # len_batch = stop_batch
    face_embeddings = []
    for i in range(len_batch):
        face_embeddings.append(torch.load(save_path / f'face_embeddings_{dataset_type}_{i}.pt'))
    face_embeddings = torch.cat(face_embeddings, dim=0)
    print(face_embeddings.shape)
    torch.save(face_embeddings, save_path / f'face_embeddings_{dataset_type}.pt')
    del face_embeddings

    pose_embeddings = []
    for i in range(len_batch):
        pose_embeddings.append(torch.load(save_path / f'pose_embeddings+{dataset_type}_{i}.pt'))
    pose_embeddings = torch.cat(pose_embeddings, dim=0)
    print(pose_embeddings.shape)
    torch.save(pose_embeddings, save_path / f'pose_embeddings_{dataset_type}.pt')
    del pose_embeddings

    text_embeddings = []
    for i in range(len_batch):
        text_embeddings.append(torch.load(save_path / f'text_embeddings_{dataset_type}_{i}.pt'))
    text_embeddings = torch.cat(text_embeddings, dim=0)
    print(text_embeddings.shape)
    torch.save(text_embeddings, save_path / f'text_embeddings_{dataset_type}.pt')
    del text_embeddings

    real_indexes = []
    for i in range(len_batch):
        real_indexes.append(torch.load(save_path / f'real_indexes_{dataset_type}_{i}.pt'))
    real_indexes = torch.cat(real_indexes, dim=0)
    print(real_indexes.shape)
    torch.save(real_indexes, save_path / f'real_indexes_{dataset_type}.pt')
    del real_indexes

save_features(dataloader, "val")

  0%|          | 0/433 [00:00<?, ?it/s]

1/1 [==============================] - 0s 95ms/step


1/1 [==============================] - 0s 130ms/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:279: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


1/1 [==============================] - 0s 85ms/step


...

1/1 [==============================] - 0s 18ms/step


 31%|███       | 135/433 [46:51<1:42:42, 20.68s/it]

1/1 [==============================] - 0s 19ms/step


 31%|███▏      | 136/433 [47:11<1:41:00, 20.40s/it]

1/1 [==============================] - ETA: 0s

In [ ]:
%ls data/saved_features

In [ ]:
%rm data/saved_features/*

In [ ]:
!du data/saved_features/* -h

81K	data/saved_features/face_embeddings_val_0.pt
81K	data/saved_features/face_embeddings_val_1.pt
81K	data/saved_features/face_embeddings_val_2.pt
81K	data/saved_features/face_embeddings_val_3.pt
81K	data/saved_features/face_embeddings_val_4.pt
81K	data/saved_features/face_embeddings_val_5.pt
81K	data/saved_features/face_embeddings_val_6.pt
481K	data/saved_features/face_embeddings_val.pt
1.0K	data/saved_features/real_indexes_val_0.pt
1.0K	data/saved_features/real_indexes_val_1.pt
1.0K	data/saved_features/real_indexes_val_2.pt
1.0K	data/saved_features/real_indexes_val_3.pt
1.0K	data/saved_features/real_indexes_val_4.pt
1.0K	data/saved_features/real_indexes_val_5.pt
1.0K	data/saved_features/real_indexes_val_6.pt
1.5K	data/saved_features/real_indexes_val.pt
49K	data/saved_features/text_embeddings_val_0.pt
49K	data/saved_features/text_embeddings_val_1.pt
49K	data/saved_features/text_embeddings_val_2.pt
49K	data/saved_features/text_embeddings_val_3.pt
49K	data/saved_features/text_embeddings

In [ ]:
save_path = project_path / 'data' / 'saved_features'
dataset_type = "val"
face_embeddings = []
for i in range(3):
    face_embeddings.append(torch.load(save_path / f'face_embeddings_{dataset_type}_{i}.pt'))
face_embeddings = torch.cat(face_embeddings, dim=0)
print(face_embeddings.shape)
torch.save(face_embeddings, save_path / f'face_embeddings_{dataset_type}_{batch_index}.pt')
del face_embeddings

NameError: ignored

#Data Loader

In [ ]:
class MSCTDDataLoader:
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    if isinstance(data, dict):
        return {k: to_device(v, device) for k, v in data.items()}
    if isinstance(data, str):
        return data
    return data.to(device)

ds = MSCTDDataSet(base_path=project_path + "data/", dataset_type = "val", load=True)
dl = DataLoader(ds, batch_size=10)
dl = MSCTDDataLoader(dl, device)
for x in dl:
  print(x)
  print(x['face_embedding'].shape)
  print(x['text_embedding'].shape)
  print(x['real_index'])
  break

In [ ]:
import torch
from torch import nn


class SimpleDenseNetwork(nn.Module):
    def __init__(self, n_classes, embedding_dimension):
        super(SimpleDenseNetwork, self).__init__()

        self.n_classes = n_classes
        self.embedding_dimension = embedding_dimension

        self.fc = nn.Sequential(
            nn.Linear(
                in_features=self.embedding_dimension,
                out_features=512,
            ),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=512, out_features=128),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=128, out_features=3),
            nn.ReLU(inplace=True),
            nn.Softmax(),
        )

    def forward(self, input_batch):
        x = input_batch
        x = self.fc(x)
        output_batch = x

        return output_batch

#Train

In [ ]:
BATCH_SIZE = 32
num_workers = 1
EPOCHS = 1
embedding_dimension = 2048
learning_rate = 0.1
momentum = 0.9
data_size = 1000

In [ ]:
import torch.optim as optim
from datetime import datetime
from torch.utils.data import DataLoader
from transformers import AutoTokenizer


def train_epoch(epoch_index, model, dataloader, loss_fn, optimizer):
    running_loss = 0.0
    last_loss = 0.0

    for data_pair_index, batch in enumerate(dataloader):
        print("--------------", data_pair_index, "-------------")
        text_embedding = batch["text_embedding"]
        face_embedding = batch["face"]
        # pose_embeding = batch["pose_embeding"]
        labels = batch["sentiment"]
        optimizer.zero_grad()

        outputs = model(torch.cat((text_embedding, face_embedding), 1))

        loss = loss_fn(outputs, labels)
        loss.backward()

        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if data_pair_index % 1000 == 999:
            last_loss = running_loss / 1000  # loss per batch
            print("  batch {} loss: {}".format(data_pair_index + 1, last_loss))
            tb_x = epoch_index * len(dataloader) + data_pair_index + 1
            print("Loss/train", last_loss, tb_x)
            running_loss = 0.0

    return last_loss


def train_model(model, epochs, train_dataloader, val_dataloader):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    for epoch in trange(epochs):
        model.train()
        train_epoch(epoch, model, train_dataloader, loss_fn, optimizer)
        model.eval()
        validate(model, val_dataloader, loss_fn)

    return model

In [ ]:
# model = SimpleDenseNetwork(n_classes=3, embedding_dimension=embedding_dimension).to(device=device)

In [ ]:
val_dataset = MSCTDDataSet(data_size, project_path + "data/", "val")
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
val_dataloader = MSCTDDataLoader(val_dataloader, device)

test_dataset = MSCTDDataSet(5000, project_path + "data/", "test")
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
test_dataloader = MSCTDDataLoader(test_dataloader, device)

/root/.hsemotions/enet_b0_8_best_afew.pt Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
/root/.hsemotions/enet_b0_8_best_afew.pt Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [ ]:
model = train_model(model, EPOCHS, val_dataloader, test_dataloader)

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:276: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


-------------- 0 -------------


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


-------------- 1 -------------
-------------- 2 -------------
-------------- 3 -------------
-------------- 4 -------------


#Evaluating

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")


def validate(model, dataloader, loss_fn):
    running_loss = 0.0
    last_loss = 0.0

    for data_pair_index, batch in enumerate(dataloader):
        print("--------------", data_pair_index, "-------------")
        text_embedding = batch["text_embedding"]
        face_embedding = batch["face"]
        # pose_embeding = batch["pose_embeding"]
        labels = batch["sentiment"]

        logits = model(torch.cat((text_embedding, face_embedding), 1))
        # print(outputs)
        accuracy.add_batch(predictions=logits.argmax(dim=1), references=labels)
        precision.add_batch(predictions=logits.argmax(dim=1), references=labels)
        loss = loss_fn(logits, labels)
        running_loss += loss.item()
        # print(running_loss)
        # print('true answer',labels)
        # print('prediction',logits.argmax(dim=1))
        # if data_pair_index==2:
        #   break
    print(accuracy.compute())
    print(precision.compute(average=None))

In [ ]:
validate(model, test_dataloader, nn.CrossEntropyLoss())

-------------- 0 -------------
-------------- 1 -------------
-------------- 2 -------------
-------------- 3 -------------
-------------- 4 -------------
-------------- 5 -------------
-------------- 6 -------------
-------------- 7 -------------
-------------- 8 -------------
-------------- 9 -------------
-------------- 10 -------------
-------------- 11 -------------
-------------- 12 -------------
-------------- 13 -------------
-------------- 14 -------------
-------------- 15 -------------
-------------- 16 -------------
-------------- 17 -------------
-------------- 18 -------------
-------------- 19 -------------
-------------- 20 -------------
-------------- 21 -------------
-------------- 22 -------------
-------------- 23 -------------
-------------- 24 -------------
-------------- 25 -------------
-------------- 26 -------------
-------------- 27 -------------
-------------- 28 -------------
-------------- 29 -------------
-------------- 30 -------------
-------------- 31 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
